In [1]:
import numpy as np
import mne
from mne.io import concatenate_raws, read_raw_fif
import mne.viz
from os import walk

In [10]:
files = []
path = '../../study1/study1_eeg/'
for (dirpath, dirnames, filenames) in walk(path):
    new_names = [dirpath+f for f in filenames if ('P-01_' in f)]
    files.extend(new_names)
    break
print(len(files))

484


In [103]:
# Read the CSV file as a NumPy array
tmp = np.loadtxt(files[0], delimiter=',')
n_channels = tmp.shape[0]
n_times = tmp.shape[1]
files= files[:5]
participant_data = np.ndarray((len(files),n_channels,n_times))

for trial in range(1,len(files)):
    new_data = np.loadtxt(files[trial], delimiter=',')
    if trial == 1:
        print('n_channels, n_times: ' + str(data.shape))
    participant_data[trial] = new_data

print('Number of epochs: ' + str(participant_data.shape))

n_channels, n_times: (64, 384)
Number of epochs: (5, 64, 384)


In [104]:
#Extract evens from file names
epochs_events = []
for f in files:
    res = f.split('_')
    epochs_events.append(res[-2])


In [126]:
unique_events = list(set(epochs_events))
unique_events_num = [i for i in range(len(unique_events))]
epoch_events_num = []

for i in range(len(epochs_events)):
    for j in range(len(unique_events)):
        if epochs_events[i] == unique_events[j]:
            epoch_events_num.append(unique_events_num[j])

print(epochs_events)
epoch_events_num = np.asanyarray(epoch_events_num)
print(epoch_events_num.shape)

['FP', 'FP', 'FP', 'FN', 'NN']
(5,)


In [105]:
n_channels = 64
ch_names = []
# Some information about the channels
for i in range(n_channels):
    ch_names.append('CH '+ str(i+1))

# Sampling rate of the Nautilus machine
sfreq = 256  # Hz

In [106]:
print(ch_names)

['CH 1', 'CH 2', 'CH 3', 'CH 4', 'CH 5', 'CH 6', 'CH 7', 'CH 8', 'CH 9', 'CH 10', 'CH 11', 'CH 12', 'CH 13', 'CH 14', 'CH 15', 'CH 16', 'CH 17', 'CH 18', 'CH 19', 'CH 20', 'CH 21', 'CH 22', 'CH 23', 'CH 24', 'CH 25', 'CH 26', 'CH 27', 'CH 28', 'CH 29', 'CH 30', 'CH 31', 'CH 32', 'CH 33', 'CH 34', 'CH 35', 'CH 36', 'CH 37', 'CH 38', 'CH 39', 'CH 40', 'CH 41', 'CH 42', 'CH 43', 'CH 44', 'CH 45', 'CH 46', 'CH 47', 'CH 48', 'CH 49', 'CH 50', 'CH 51', 'CH 52', 'CH 53', 'CH 54', 'CH 55', 'CH 56', 'CH 57', 'CH 58', 'CH 59', 'CH 60', 'CH 61', 'CH 62', 'CH 63', 'CH 64']


In [127]:
%matplotlib inline
# Create the info structure needed by MNE
epochs_info = mne.create_info(ch_names, sfreq)

# Finally, create the Raw object
print(participant_data.ndim)
epochs = mne.EpochsArray(data=participant_data, info=epochs_info, events=epoch_events_num)


3


IndexError: too many indices for array

In [123]:
epochs.average()

ValueError: picks (array-like of str) could not be interpreted as channel names (no channel "data_or_ica"), channel types (no type "data_or_ica"), or a generic type (just "all" or "data")